# Attribution

This notebook is a modified version of one shared with us by Martin Jones of the Francis Crick Institute. Thank you, Martin!

## REQUIRED: Set up Zooniverse project information

Set variables that define the Zooniverse project and user and image directory containing jpegs to upload.

In [ ]:
# unique zooniverse project id
PROJECT_ID = "" # get this info from your zooniverse account

# whether uploaded images are flipbooks of a few jpegs or a single jpeg
IS_FLIPBOOK = True

# only applies to flipbooks (ignored otherwise): number of
# images above and below the central slice
# of the flipbook
# e.g. SPAN = 2 is the central slice plus 2 from above and 2 from below (a total of 5 images)
# CAUTION: CHANGING THIS NUMBER MAY BREAK OTHER SCRIPTS!
SPAN = 2

# zooniverse username
USERNAME = ""

# name of the subject set to be created
SUBJECT_SET_NAME = ""

# whether this is a test if True, then
# subjects won't be uploaded to zooniverse
TESTING = False

# local directory containing jpeg images for upload
wdir = ""

# run checks
assert PROJECT_ID, "Must provide a Zooniverse project ID"
assert USERNAME, "Must provide a Zooniverse username"
assert SUBJECT_SET_NAME, "Must provide a name for the subject set"
assert WDIR, "Must provide a directory containing jpeg images"

if IS_FLIPBOOK:
    assert SPAN > 1

## Upload images

Login to Zooniverse, create the new subject set, add images/flipbooks and upload!

In [ ]:
from panoptes_client import Project, Panoptes, Subject, SubjectSet
import glob
import os
import sys
import getpass
import re
from tqdm import tqdm

In [ ]:
# This function builds the subject from the chosen set of images and attaches metadata
def build_subject_flipbook(project, file_list, start_index, end_index):
    """
    Use this function to add flipbooks as subjects 
    to a Zooniverse project.
    """
    subject = Subject() # Inititialise a subject
    subject.links.project = project # ...attach it to a project
    subject.metadata['Subject ID'] = start_index + SPAN # Add the names of the images
    
    # For loop to attach the images to the subject one-by-one
    for i, idx in enumerate(range(start_index, end_index)):
        fname = str(file_list[idx])
        subject.add_location(fname)
        subject.metadata['Image %d' % i] = os.path.basename(fname)
        
    subject.metadata['default_frame'] = SPAN + 1  # We want people to annotate the middle image
    
    # Metadata from here should be changed according to the data
    # any keys are allowed for the metadata
    subject.metadata['Microscope'] = '' # give a name to the microscope
    subject.metadata['Raw XY resolution (nm)'] = 1
    subject.metadata['Raw Z resolution (nm)'] = 1
    subject.metadata['Scaling factor'] = 1
    subject.metadata['jpeg quality (%)'] = 100
    subject.metadata['Attribution'] = '' # reference the source of this data
    subject.metadata['Description'] = '' # describe the dataset being uploaded
    print("Starting to save")
    print(subject)
    subject.save()
    print("Subject saved")

    return subject

def build_subject_image(project, file_list, index):
    """
    Use this function to add individual images (not flipbooks) 
    as subjects to a Zooniverse project.
    """
    subject = Subject() # Inititialise a subject
    subject.links.project = project # ...attach it to a project
    subject.metadata['Subject ID'] = index
    
    # For loop to attach the images to the subject one-by-one
    fname = str(file_list[index])
    subject.add_location(fname)
    subject.metadata['Image 0'] = os.path.basename(fname)
    
    # Metadata from here should be changed according to the data
    # any keys are allowed for the metadata
    subject.metadata['Microscope'] = '' # give a name to the microscope
    subject.metadata['Raw XY resolution (nm)'] = 1
    subject.metadata['Raw Z resolution (nm)'] = 1
    subject.metadata['Scaling factor'] = 1
    subject.metadata['jpeg quality (%)'] = 100
    subject.metadata['Attribution'] = '' # reference the source of this data
    subject.metadata['Description'] = '' # describe the dataset being uploaded
    print("Starting to save")
    print(subject)
    subject.save()
    print("Subject saved")

    return subject

This function connects to the Zooniverse, make sure the Project ID and username were set correctly earlier.

In [ ]:
def connect_to_zooniverse(project_id=PROJECT_ID, user_name=USERNAME):
    try:
        password = getpass.getpass(prompt='Password: ', stream=None)
        Panoptes.connect(username=user_name, password=password)
        print("Connected to Zooniverse")
    except Exception as e:
        print("Couldn't connect to Zooniverse")
        print("Exception {}".format(e))
        sys.exit(1)
        
    print(f"Connecting to {project_id}...")
    project = Project.find(slug=project_id)
    print("...connected!")
    return project

In [ ]:
# Helper function to initialise a "subject set" and attach it to a project
def initialise_subject_set(project, subject_name):
    subject_set = SubjectSet()
    subject_set.links.project = project
    subject_set.display_name = subject_name
    subject_set.save()
    return subject_set

# Function to build a subject set from a fixed range of images
def build_subject_set_flipbook(project, file_list, file_idx_start, file_idx_stop):
    print(f"project {project}\n", 
          f"file_idx_start {file_idx_start}\n", 
          f"file_idx_stop {file_idx_stop}\n"
         )
    
    print(f"Building subject set from files {file_idx_start}-{file_idx_stop}")
    subjects = []
    
    subject = build_subject_flipbook(project, file_list, file_idx_start, file_idx_stop)
    subjects.append(subject)
    
    return subjects

def build_subject_set_image(project, file_list, file_idx):
    print(f"project {project}\n", 
          f"file_idx {file_idx}\n"
         )
    
    print(f"Building subject set from file {file_idx}")
    subjects = []

    subject = build_subject_image(project, file_list, file_idx)
    subjects.append(subject)
    
    return subjects

Connect to the Zooniverse using your credentials

In [ ]:
project = connect_to_zooniverse(project_id=PROJECT_ID)

In [ ]:
# This code will work for directories populated with jpg images
# by either the prep_stacks.py or prep_images.py script

file_list = sorted(glob.glob(os.path.join(wdir, '*.jpg')))
n_files = len(file_list)

file_basenames = [os.path.basename(file) for file in file_list]
if IS_FLIPBOOK:
    # the the key '_zloc' is needed to organize flipbook images in the
    # correct order (added correctly by the prep_stacks.py script)
    im_per_fb = 2 * SPAN + 1 # number of images in each flipbook
    file_basename_prefixes = [file.split('_zloc')[0] for file in file_basenames]
    indices = list(range(0, len(file_list) + 1, im_per_fb))
    print('num flipbooks:', n_files / im_per_fb)
else:
    file_basename_prefixes = [file.split('.jpg')[0] for file in file_basenames]
    indices = list(range(0, len(file_list)))
    print('num images:', n_files)

**Note: this can be a very slow process to upload!**

In [ ]:
# create subject set of name defined above
print(f"Creating subject set name {SUBJECT_SET_NAME}\n")
subject_set = initialise_subject_set(project, SUBJECT_SET_NAME)

In [ ]:
# add images/flipbooks as subjects to the subject set
subjects = []
if IS_FLIPBOOK:
    for counter, (start_index, end_index) in enumerate(zip(indices[:-1], indices[1:])):
        print(f"\n*******\nStep {counter}")
        subjects.append(build_subject_set_flipbook(project, file_list, start_index, end_index))
else:
    for counter, index in enumerate(indices):
        print(f"\n*******\nStep {counter}")
        subjects.append(build_subject_set_image(project, file_list, index))

In [ ]:
# upload the subject sets to the Zooniverse
if not TESTING:
    for subj in tqdm(subjects):
        subject_set.add(subj)

## Add subject IDs

Optionally add subject ids to each subjects metadata. This makes it possible for annotators to lookup and reference particular subjects for later review.

In [ ]:
# load all subject sets in project and print their metadata
subject_sets = SubjectSet.where(scope='project', project_id=project.id)
ssets = list(subject_sets)

for i, sset in enumerate(ssets):
    print(i, sset.raw)

In [ ]:
# pick the subject set you want to add subject_ids to
subject_set = ssets[2]

In [ ]:
# print and confirm subject set details
subject_set

In [ ]:
# get the subjects for one of the subject sets
# (index 0 for illustration only)
subjects = list(ssets[0].subjects)

In [ ]:
# to add ID key to the subject metadata
# uncomment the 3 lines below

#for subject in tqdm(subjects):
#    subject.metadata['ID'] = subject.raw['id']
#    subject.save()

## Drop subject

Delete subjects that were created accidentally.

In [ ]:
# load all subject sets in project and print their metadata
subject_sets = SubjectSet.where(scope='project', project_id=project.id)
ssets = list(subject_sets)

for i, sset in enumerate(ssets):
    print(i, sset.raw)

In [ ]:
# get the subjects for one of the subject sets
# (index 0 for illustration only)
subjects = list(ssets[0].subjects)

In [ ]:
# delete the subjects!! use with care!!
# uncomment the 2 lines below

#for subject in tqdm(subjects):
#    subject.delete()